In [1]:
import numpy as np
import pandas as pd
np.random.seed(8)
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

from keras.models import Model
from keras.layers import Input, Embedding, Dense, Conv2D, MaxPool2D, Conv1D, MaxPooling1D, GlobalMaxPooling1D 
from keras.layers import Reshape, Flatten, Concatenate, Dropout, SpatialDropout1D
from keras.preprocessing import text, sequence
from keras.callbacks import Callback

import warnings
warnings.filterwarnings('ignore')

#import os
#os.environ['OMP_NUM_THREADS'] = '4'

train = pd.read_csv('train.csv')
test_cm = pd.read_csv('test.csv')
test_lb = pd.read_csv('test_labels.csv')
test_all = pd.merge(test_cm, test_lb, on='id')
test = test_all[test_all['toxic'] != -1]

Using TensorFlow backend.


In [2]:
X_train = train["comment_text"].values
y_train = train[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]].values
X_test = test["comment_text"].values
y_test = test[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]].values

In [3]:
vocab = 100000
maxlen = 200
embed_size = 300

In [4]:
t = text.Tokenizer(num_words=vocab)
t.fit_on_texts(list(X_train))

In [5]:
X_train = t.texts_to_sequences(X_train)
X_test = t.texts_to_sequences(X_test)
x_train = sequence.pad_sequences(X_train, maxlen=maxlen)
x_test = sequence.pad_sequences(X_test, maxlen=maxlen)

In [6]:
EMBEDDING_FILE = 'crawl-300d-2M.vec'
embeddings_index = dict()
f = open(EMBEDDING_FILE)
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s pre-trained words' % len(embeddings_index))


Loaded 2000000 pre-trained words


In [7]:
# create a weight matrix for words in training docs
embedding_matrix = np.zeros((vocab, embed_size))
for word, i in t.word_index.items():
    if i >= vocab: 
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [8]:
def make_model(activation, filters):
    #activation='relu'
    #num_filters = 32

    inp = Input(shape=(maxlen, ))
    x = Embedding(vocab, embed_size, weights=[embedding_matrix], input_length=maxlen, trainable=False)(inp)
    x = SpatialDropout1D(0.4)(x)
    x = Reshape((maxlen, embed_size, 1))(x)

    conv_0 = Conv2D(filters, kernel_size=(2, embed_size), activation=activation)(x)
    maxpool_0 = MaxPool2D(pool_size=(maxlen - 1, 1))(conv_0)
    #conv_1 = Conv2D(num_filters, kernel_size=(3, embed_size), activation=activation)(x)
    #maxpool_1 = MaxPool2D(pool_size=(maxlen - 2, 1))(conv_1)
    #conv_2 = Conv2D(num_filters, kernel_size=(4, embed_size), activation=activation)(x)
    #maxpool_2 = MaxPool2D(pool_size=(maxlen - 3, 1))(conv_2)
    #conv_3 = Conv2D(num_filters, kernel_size=(5, embed_size), activation=activation)(x)
    #maxpool_3 = MaxPool2D(pool_size=(maxlen - 4, 1))(conv_3)
    #conv_4 = Conv2D(num_filters, kernel_size=(6, embed_size), activation=activation)(x)
    #maxpool_4 = MaxPool2D(pool_size=(maxlen - 5, 1))(conv_4)

    #y = Concatenate(axis=1)([maxpool_0, maxpool_1, maxpool_2, maxpool_3, maxpool_4])
    #y = Flatten()(y)
    y = Flatten()(maxpool_0)
    y = Dropout(0.1)(y)

    outp = Dense(6, activation="sigmoid")(y)

    model = Model(inputs=inp, outputs=outp)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model
#model = make_model()
#model.summary()

In [9]:
from sklearn.model_selection import ShuffleSplit
cv = ShuffleSplit(n_splits = 1, test_size = 0.1)

In [10]:
class RocAuc(Callback):

    def on_epoch_end(self, epoch, logs={}):
        y_pred = self.model.predict(self.validation_data[0], verbose=0)
        score = roc_auc_score(self.validation_data[1], y_pred)
        print("\n roc_auc - epoch: %d - score: %.6f \n" % (epoch+1, score))

In [11]:
from keras.callbacks import ModelCheckpoint
batch_size=256
checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.pre_trained.conv2.testscore.hdf5', verbose=1, save_best_only=True)
RocAuc = RocAuc()

In [12]:
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
my_classifier = KerasClassifier(make_model, batch_size=256)
# define the grid search parameters
#activation = ['relu', 'elu', 'tanh']
#param_grid = dict(activation=activation)
#param_grid = {'activation' : activation}
validator = GridSearchCV(my_classifier, param_grid={'activation': ['relu', 'elu', 'tanh'],
                                                   'epochs': [3],
                                                   'filters': [32]},
                                                   cv = cv, 
                                                   scoring = 'roc_auc', 
                                                   n_jobs=1)

In [13]:
grid_result = validator.fit(x_train, y_train)

Epoch 1/3
143613/143613 [==============================] - 189s 1ms/step - loss: 0.1206 - acc: 0.9685
Epoch 2/3
143613/143613 [==============================] - 186s 1ms/step - loss: 0.0660 - acc: 0.9780
Epoch 3/3
143613/143613 [==============================] - 186s 1ms/step - loss: 0.0608 - acc: 0.9791
Epoch 1/3
143613/143613 [==============================] - 200s 1ms/step - loss: 0.1113 - acc: 0.9696
Epoch 2/3
143613/143613 [==============================] - 202s 1ms/step - loss: 0.0606 - acc: 0.9793
Epoch 3/3
143613/143613 [==============================] - 200s 1ms/step - loss: 0.0557 - acc: 0.9800
Epoch 1/3
143613/143613 [==============================] - 194s 1ms/step - loss: 0.1424 - acc: 0.9642
Epoch 2/3
143613/143613 [==============================] - 193s 1ms/step - loss: 0.0602 - acc: 0.9790
Epoch 3/3
143613/143613 [==============================] - 193s 1ms/step - loss: 0.0543 - acc: 0.9801
Epoch 1/3
159571/159571 [==============================] - 221s 1ms/step - loss: 0

In [14]:
print('The parameters of the best model are: ')
print(validator.best_params_)

# validator.best_estimator_ returns sklearn-wrapped version of best model.
# validator.best_estimator_.model returns the (unwrapped) keras model
best_model = validator.best_estimator_.model
metric_names = best_model.metrics_names
metric_values = best_model.evaluate(x_test, y_test)
for metric, value in zip(metric_names, metric_values):
    print(metric, ': ', value)

The parameters of the best model are: 
{'activation': 'elu', 'epochs': 3, 'filters': 32}
63978/63978 [==============================] - 25s 398us/step
loss :  0.06913161541262879
acc :  0.9723498691313004


In [15]:
y_pred = best_model.predict(x_test, batch_size=256)

In [16]:
score = roc_auc_score(y_test, y_pred)
print("\n roc_auc score: %.6f \n" % (score))


 roc_auc score: 0.975425 



In [ ]:
t